In [107]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
# Today price: https://selectra.es/energia/info/que-es/precio-kwh
# Tomorrow price: https://selectra.es/energia/info/que-es/precio-kwh#precio-luz-manana
# MoM evolution: https://selectra.es/energia/info/que-es/precio-kwh#precio-luz-hoy

# Overall metrics: https://tarifaluzhora.es/
# Electro devices per year: https://tarifaluzhora.es/info/consumo-mensual-luz#como-reparte-consumo

# Consumption per device per year: https://comparadorluz.com/faq/consumo-medio-casa#consumo-luz-electrodomesticos
# Consumption per device: https://comparadorluz.com/faq/consumo-medio-casa#consumo-medio-luz

url = 'https://tarifaluzhora.es/'

In [3]:
headers= {'Accept-Encoding':'gzip, deflate',
          'Accept-Language':'en-US,en;q=0.9',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}

response = requests.get(url, headers = headers)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content)
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="es" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <link href="https://tarifaluzhora.es/" rel="canonical"/>
  <link href="https://tarifaluzhora.es/" rel="shortlink"/>
  <meta content="Comprueba aquí el Precio actual de la Luz | Precios máximos y mínimos de la luz a lo largo del día, con y sin Discriminación Horaria del día que quieras" name="description"/>
  <meta content="ES" property="og:country_name"/>
  <meta content="Drupal 9 (https://www.drupal.org)" name="Generator"/>
  <meta content="width" name="MobileOptimized"/>
  <meta content="true" name="HandheldFriendly"/>
  <meta content="widt

In [87]:
average_today_price = soup.find('div',attrs={'class':'inner_block gauge_day'})
average_today_price = average_today_price.span.string[:6]

cheapest_today_time = soup.find('div',attrs={'class':'inner_block gauge_low'})
cheapest_today_time = cheapest_today_time.span.string.replace('h','')

expensive_today_time = soup.find('div',attrs={'class':'inner_block gauge_hight'})
expensive_today_time = expensive_today_time.span.string.replace('h','')

lowest_today_price = soup.find('span',attrs={'class':'sub_text green'})
lowest_today_price= lowest_today_price.get_text().replace('\t','').replace('\n','')[:7]

highest_today_price = soup.find('span',attrs={'class':'sub_text red'})
highest_today_price= highest_today_price.get_text().replace('\t','').replace('\n','')[:7]

print(f'Average price of today is: {average_today_price}€ per kwh')
print(f'Lowest price of today is: {lowest_today_price}€ per kwh')
print(f'Highest price of today is: {highest_today_price}€ per kwh')
print(f'Cheapest time of today was between {cheapest_today_time} hs')
print(f'Expensive time of today was between {expensive_today_time} hs')

Average price of today is: 0.3063€ per kwh
Lowest price of today is: 0.25017€ per kwh
Highest price of today is: 0.38338€ per kwh
Cheapest time of today was between 03 - 04 hs
Expensive time of today was between 20 - 21 hs


In [86]:
hour_range = soup.find_all('span',attrs={'itemprop':'description'})
hour_price = soup.find_all('span',attrs={'itemprop':'price'})

In [109]:
hours = []

for i in hour_range:
    hours.append(i.string.replace(':','').replace('h',''))
    
prices = []

for i in hour_price:
    prices.append(i.string.replace('€/kW','').replace('h','').strip())

In [121]:
price_per_hour = pd.DataFrame()  
price_per_hour['Hours'] = hours
price_per_hour['Prices'] = prices
price_per_hour['Prices'] = price_per_hour['Prices'].astype(float)
price_per_hour['Avg Today'] = price_per_hour['Prices'].mean()
price_per_hour['Deviation'] = price_per_hour['Prices'].mean() - price_per_hour['Prices']

def price_flag(x):
    if x<0:
        return 'Expensive'
    elif x>0: 
        return 'Cheaper'
    else:
        return 'Regular'

price_per_hour['Price Flag'] = price_per_hour['Deviation'].apply(price_flag)

def light_consumption(x):
    return x*0.117

def kitchen_consumption(x):
    return x*0.093

def hot_water_consumption(x):
    return x*0.075

def heat_consumption(x):
    return x*0.074

def refrigeration_consumption(x):
    return x*0.023

def standby_consumption(x):
    return x*0.066

def home_appliances_consumption(x):
    return x*0.552

price_per_hour['Home Appliances'] = price_per_hour['Prices'].apply(home_appliances_consumption)
price_per_hour['Light'] = price_per_hour['Prices'].apply(light_consumption)
price_per_hour['Kitchen'] = price_per_hour['Prices'].apply(kitchen_consumption)
price_per_hour['Hot water'] = price_per_hour['Prices'].apply(hot_water_consumption)
price_per_hour['Heat'] = price_per_hour['Prices'].apply(heat_consumption)
price_per_hour['Refrigeration'] = price_per_hour['Prices'].apply(refrigeration_consumption)
price_per_hour['Stand By'] = price_per_hour['Prices'].apply(standby_consumption)

price_per_hour

,Hours,Prices,Avg Today,Deviation,Price Flag,Home Appliances,Light,Kitchen,Hot water,Heat,Refrigeration,Stand By
0,00 - 01,0.26338,0.306302,0.042922,Cheaper,0.145386,0.030815,0.024494,0.019754,0.019490,0.006058,0.017383
1,01 - 02,0.25911,0.306302,0.047192,Cheaper,0.143029,0.030316,0.024097,0.019433,0.019174,0.005960,0.017101
2,02 - 03,0.25121,0.306302,0.055092,Cheaper,0.138668,0.029392,0.023363,0.018841,0.018590,0.005778,0.016580
3,03 - 04,0.25017,0.306302,0.056132,Cheaper,0.138094,0.029270,0.023266,0.018763,0.018513,0.005754,0.016511
4,04 - 05,0.25275,0.306302,0.053552,Cheaper,0.139518,0.029572,0.023506,0.018956,0.018703,0.005813,0.016681
5,05 - 06,0.26680,0.306302,0.039502,Cheaper,0.147274,0.031216,0.024812,0.020010,0.019743,0.006136,0.017609
6,06 - 07,0.30294,0.306302,0.003362,Cheaper,0.167223,0.035444,0.028173,0.022720,0.022418,0.006968,0.019994
7,07 - 08,0.31440,0.306302,-0.008098,Expensive,0.173549,0.036785,0.029239,0.023580,0.023266,0.007231,0.020750
8,08 - 09,0.36044,0.306302,-0.054138,Expensive,0.198963,0.042171,0.033521,0.027033,0.026673,0.008290,0.023789
9,09 - 10,0.33417,0.306302,-0.027868,Expensive,0.184462,0.039098,0.031078,0.025063,0.024729,0.007686,0.022055
